# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.129451e+02     1.581398e+00
 * time: 0.7261879444122314
     1     1.091651e+01     8.547808e-01
 * time: 1.9987280368804932
     2    -1.176096e+01     9.355712e-01
 * time: 2.616676092147827
     3    -3.393286e+01     6.864485e-01
 * time: 3.482422113418579
     4    -4.762610e+01     5.204062e-01
 * time: 4.361424922943115
     5    -5.699029e+01     1.825766e-01
 * time: 5.241307020187378
     6    -5.990645e+01     1.127285e-01
 * time: 5.822336912155151
     7    -6.094197e+01     4.596010e-02
 * time: 6.402735948562622
     8    -6.130848e+01     7.805175e-02
 * time: 6.998521089553833
     9    -6.161226e+01     2.898982e-02
 * time: 7.5784900188446045
    10    -6.178957e+01     2.524730e-02
 * time: 8.160737037658691
    11    -6.194642e+01     2.172310e-02
 * time: 8.742638111114502
    12    -6.201811e+01     2.000961e-02
 * time: 9.332704067230225
    13    -6.209834e+01     1.769683e-02
 * time: 9.913828134536743
    

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671068
    AtomicLocal         -18.8557684
    AtomicNonlocal      14.8522651
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666460466